# Spotify Song Suggester

## Team Members
- Sara West, DSPT8 Unit 4
- Yeraldina Estrella, DSPT9 Unit 3
- Eugene Sheely, DSPT9 Unit 3

## Project Description
A user will input some basic information about a song enjoyed. The song suggester will evaluate the song and find 5 similar songs the user may also enjoy.

In [ ]:
# Import required packages
import numpy as np
import pandas as pd
import tensorflow as tf

print(f'Numpy Version: {np.__version__}')
print(f'Pandas Version: {np.__version__}')
print(f'Tensorflow Version: {tf.__version__}')

Numpy Version: 1.19.5
Pandas Version: 1.19.5
Tensorflow Version: 2.4.1


In [ ]:
!pip install sklearn
!pip install --upgrade category_encoders

     |████████████████████████████████| 81kB 5.7MB/s 


In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing
from sklearn.neighbors import NearestNeighbors
import category_encoders as ce 

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)

## Load Data

In [ ]:
# from google.colab import drive, files
# uploaded = files.upload()
# data files too large to store on GitHub
from google.colab import drive 
drive.mount("/content/drive")

Mounted at /content/drive


###Primary Dataset, Songs

In [ ]:
# Load dataset
path ='/content/drive/My Drive/Data/spotify_data.csv'
data = pd.read_csv(path)
print(data.shape)
data.sample(4)

(174389, 19)


,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
156551,0.0975,['The Jompson Brothers'],0.4740,290973,0.317,0,2RESEfP5xCZFWl4BSIHpQC,0.000319,11,0.0657,-7.336,0,Barely Alive,49,2010-11-30,0.0318,113.144,0.247,2010
133803,0.8720,['Queen'],0.3660,78667,0.398,0,0Smf2LIhfokonnV8YEqp6b,0.000002,5,0.1300,-11.499,1,Nevermore - Remastered 2011,20,1974-03-08,0.0367,118.480,0.186,1974
98154,0.8630,"['Richard Rodgers', 'Mormon Tabernacle Choir', 'Arthur Harris', 'Columbia Symphony Orchestra']",0.0923,123973,0.210,0,3wL9QQRtlxKWvYwgDbbdli,0.005680,4,0.1280,-18.572,1,"Oh, What A Beautiful Mornin'",14,1959,0.0425,69.377,0.194,1959
84654,0.3020,['Gram Parsons'],0.6010,163440,0.828,0,6p7g9kd8ZgDyk05xJgxB8C,0.000057,9,0.2020,-8.321,1,Still Feeling Blue - Remastered Version,31,1973,0.0497,106.828,0.953,1973


In [ ]:
data.drop(columns=['release_date'], inplace=True)
print(data.shape)
data.sample(4)

(174389, 18)


,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,speechiness,tempo,valence,year
92389,0.000666,['Julian Jeweil'],0.808,368640,0.698,0,4RlJdPh1FCw22ncsqozx8O,0.875000,1,0.0925,-7.991,1,Mad (Original Mix) - Best Of 2013,0,0.0735,125.010,0.262,2013
735,0.996000,['Francisco Canaro'],0.494,176747,0.232,0,1gxCV37YTTaWCPACNYdImn,0.927000,9,0.1740,-18.142,0,Juventud - Remasterizado,0,0.1750,65.900,0.541,1924
106016,0.000157,['Metallica'],0.331,397533,0.782,0,6FCSwETzhnykKYJLIMTewZ,0.010600,3,0.1460,-5.624,0,Astronomy,38,0.0413,85.741,0.157,1998
82670,0.803000,"['Dinah Washington', 'Malcolm Addey']",0.536,145973,0.187,0,4caVIPXaKAmEyPIpTqKWW3,0.000034,0,0.1190,-13.270,1,Don't Come Running Back to Me,19,0.0396,67.491,0.408,1963


In [ ]:
#check for exact duplicate rows
data.duplicated().sum()

2159

In [ ]:
data.drop_duplicates(inplace=True)
print(data.shape)
print(data.id.nunique()) #verify only unique song id's remain

(172230, 18)
172230


In [ ]:
data.sample(4)

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,speechiness,tempo,valence,year
120307,0.7510,"['Aram Khachaturian', 'London Symphony Orchestra', 'Stanley Black']",0.414,151760,0.621,0,672qK11fprMeMxDK9Fb0NU,0.873000,0,0.1160,-12.607,1,Gayaneh: Sabre Dance,30,0.0390,85.753,0.631,1989
129657,0.0217,['PK Farstad'],0.645,224973,0.540,0,1hUEgzbP5Bx9vRFlUPccmF,0.824000,0,0.1570,-9.753,1,Ebb Tide,0,0.0280,91.992,0.301,1953
135596,0.1610,['Three Dog Night'],0.616,216227,0.911,0,1juyGCItoK8gYro94faicl,0.000000,2,0.3740,-6.398,1,Joy To The World - Edit,28,0.0309,128.700,0.970,1983
107187,0.8410,['Renee Olstead'],0.666,212160,0.303,0,4bJbSERtCqK9D6DRUPMmMh,0.000711,11,0.0971,-11.028,0,A Love That Will Last,57,0.0356,81.923,0.342,2004


In [ ]:
# view one song with multiple artists
data.iloc[62729]

acousticness                                                                                   0.919
artists             ['Richard Adler', 'Jerry Ross', 'The Original Broadway Cast Of The Pajama Game']
danceability                                                                                   0.595
duration_ms                                                                                   106440
energy                                                                                         0.139
explicit                                                                                           0
id                                                                            6kfWm0913KSuaYy8uTmWIX
instrumentalness                                                                                   0
key                                                                                                0
liveness                                                                                   

In [ ]:
data.artists.nunique()

36195

In [ ]:
# reduce artists column to only the first artist listed
data['artists'] = data['artists'].apply(lambda x: x.split(',',1)[0])
data['artists'] = data['artists'].apply(lambda x: x.strip("[']"))
data.artists.nunique()

23169

In [ ]:
# verify artist column is as expected
data.iloc[62729,1]

'Richard Adler'

In [ ]:
data.dtypes

acousticness        float64
artists              object
danceability        float64
duration_ms           int64
energy              float64
explicit              int64
id                   object
instrumentalness    float64
key                   int64
liveness            float64
loudness            float64
mode                  int64
name                 object
popularity            int64
speechiness         float64
tempo               float64
valence             float64
year                  int64
dtype: object

### Genres dataset

In [ ]:
# genres per artist
path ='/content/drive/My Drive/Data/data_w_genres.csv'
data_g = pd.read_csv(path)
print(data_g.shape)
data_g.sample(4)

(32539, 16)


,artists,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,count,genres
8815,Eric Burdon,0.311460,0.667067,407213.4,0.477207,0.003359,0.126973,-12.599533,0.084567,117.568733,0.659547,29.533333,2,0,15,"['blues rock', 'british blues', 'classic rock', 'psychedelic rock']"
2436,Babes In Toyland,0.000022,0.324000,169010.6,0.833200,0.361000,0.164600,-8.714600,0.062700,145.255800,0.656800,35.600000,11,0,10,"['alternative pop', 'grunge', 'minneapolis punk', 'riot grrrl']"
15350,Kenny Garrett,0.857000,0.364000,317946.0,0.114000,0.635000,0.083900,-16.552000,0.033300,117.715000,0.057500,46.000000,1,1,2,"['bebop', 'contemporary jazz', 'contemporary post-bop', 'cool jazz', 'hard bop', 'jazz', 'jazz fusion', 'jazz saxophone']"
3566,Bobby Brown Quartet,0.707000,0.341000,392267.0,0.410000,0.083700,0.098400,-12.915000,0.032900,123.210000,0.316000,0.000000,11,0,2,[]


In [ ]:
# reduce data_g to only the pertinent columns
data_g = data_g[['artists', 'genres']]
data_g.sample(5)

,artists,genres
11320,Gustavo Gallo,[]
20994,Nothing More,['alternative metal']
2800,Bell Telephone Hour Orchestra,[]
3059,Big D,"['ccm', 'christian alternative rock', 'christian music', 'worship']"
14746,Julio Chaidez,"['banda', 'corrido', 'deep regional mexican', 'norteno', 'nuevo regional mexicano', 'regional mexican']"


In [ ]:
data_g.artists.nunique()

32539

In [ ]:
data_g.isnull().sum()

artists    0
genres     0
dtype: int64

In [ ]:
# create "not specified" genre for empty sets
data_g['genres'] = data_g['genres'].apply(lambda x: str(x).replace("[]","['not specified']"))
data_g.sample(5)

,artists,genres
11497,Hank Williams III,"['cowpunk', 'outlaw country']"
5400,Citizen Jane,['not specified']
8684,Emilio Renzi,['italian tenor']
28227,The Enoch Light Brigade,['not specified']
15492,Kidnap,"['future garage', 'new french touch', 'tropical house', 'vapor soul']"


###New Dataset, Songs with Genres

In [ ]:
df = pd.merge(data, data_g, how="left", on = ["artists", "artists"])
print(df.shape)
df.sample(4)

(172230, 19)


,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,speechiness,tempo,valence,year,genres
2395,0.63300,Tadeusz Dolega Mostowicz,0.737,131900,0.206,0,0XG7FjOJuAxaeBR0iA3zkU,0.000000,6,0.2850,-23.705,1,Chapter 10.3 - Kariera Nikodema Dyzmy,0,0.9610,59.157,0.720,1932,['not specified']
103170,0.00144,Skid Row,0.413,204507,0.880,0,5BWg5BfnMRS8hBkP6NaHYH,0.000004,9,0.3220,-10.038,1,Can't Stand the Heartache,36,0.0468,143.387,0.566,1989,"['album rock', 'glam metal', 'hard rock', 'metal', 'rock']"
93769,0.75700,Sinclair Lewis,0.711,304863,0.195,0,5RaxZwkZugHHdYl1uZUti4,0.000000,4,0.2110,-17.432,1,"Das ist bei uns nicht möglich, Kapitel 47",0,0.9540,109.172,0.431,1935,['not specified']
69850,0.54900,Harry Connick,0.714,251600,0.139,0,2LdrpTJ1Y2FKp4wcbkf0sY,0.000005,0,0.0984,-17.683,1,Let's Call the Whole Thing Off,35,0.0705,127.318,0.354,1989,NaN


In [ ]:
df.isnull().sum() # It appears that not all artists in the song list were included in the genre list

acousticness           0
artists                0
danceability           0
duration_ms            0
energy                 0
explicit               0
id                     0
instrumentalness       0
key                    0
liveness               0
loudness               0
mode                   0
name                   0
popularity             0
speechiness            0
tempo                  0
valence                0
year                   0
genres              2174
dtype: int64

In [ ]:
# fill in empty genres
df['genres'].fillna("['not specified']", inplace = True)
df.isnull().sum()

acousticness        0
artists             0
danceability        0
duration_ms         0
energy              0
explicit            0
id                  0
instrumentalness    0
key                 0
liveness            0
loudness            0
mode                0
name                0
popularity          0
speechiness         0
tempo               0
valence             0
year                0
genres              0
dtype: int64

In [ ]:
# df.explode('genres')

## Data Exploration

In [ ]:
# Descriptive statistics for key fields
df.describe()
# Add visualizations (where applicable)

,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence,year
count,172230.000000,172230.000000,1.722300e+05,172230.000000,172230.000000,172230.000000,172230.000000,172230.000000,172230.000000,172230.000000,172230.000000,172230.000000,172230.000000,172230.000000,172230.000000
mean,0.501914,0.536246,2.326718e+05,0.480989,0.067956,0.194968,5.204395,0.211396,-11.777780,0.703147,25.925913,0.105896,116.937762,0.525249,1976.582001
std,0.379394,0.175858,1.471768e+05,0.272032,0.251670,0.332974,3.517851,0.180661,5.690105,0.456873,21.866745,0.183053,30.290836,0.264392,26.708673
min,0.000000,0.000000,4.937000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,-60.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1920.000000
25%,0.091300,0.414000,1.662000e+05,0.248000,0.000000,0.000000,2.000000,0.099300,-14.935750,0.000000,1.000000,0.035200,93.834000,0.311000,1955.000000
50%,0.523000,0.547000,2.057175e+05,0.463000,0.000000,0.000498,5.000000,0.138000,-10.867000,1.000000,26.000000,0.045400,115.657000,0.537000,1977.000000
75%,0.896000,0.668000,2.656000e+05,0.708000,0.000000,0.238000,8.000000,0.270000,-7.532000,1.000000,42.000000,0.076000,134.991000,0.744000,1998.000000
max,0.996000,0.988000,5.338302e+06,1.000000,1.000000,1.000000,11.000000,1.000000,3.855000,1.000000,100.000000,0.971000,243.507000,1.000000,2021.000000


In [ ]:
df[['duration_ms', 'key', 'loudness', 'popularity', 'tempo', 'year']].describe()

,duration_ms,key,loudness,popularity,tempo,year
count,1.722300e+05,172230.000000,172230.000000,172230.000000,172230.000000,172230.000000
mean,2.326718e+05,5.204395,-11.777780,25.925913,116.937762,1976.582001
std,1.471768e+05,3.517851,5.690105,21.866745,30.290836,26.708673
min,4.937000e+03,0.000000,-60.000000,0.000000,0.000000,1920.000000
25%,1.662000e+05,2.000000,-14.935750,1.000000,93.834000,1955.000000
50%,2.057175e+05,5.000000,-10.867000,26.000000,115.657000,1977.000000
75%,2.656000e+05,8.000000,-7.532000,42.000000,134.991000,1998.000000
max,5.338302e+06,11.000000,3.855000,100.000000,243.507000,2021.000000


In [ ]:
df.describe(include=[object])

,artists,id,name,genres
count,172230,172230,172230,172230
unique,23169,172230,137013,9512
top,Francisco Canaro,4MB99qs6Uwt7pforoKGrHa,White Christmas,['not specified']
freq,2228,1,103,27923


In [ ]:
print(round(100*27923/172230, 2), '% of songs have no genre specified')

16.21 % of songs have no genre specified


In [ ]:
df['artists'] = df.artists.astype(str).apply(lambda x: x.lower())
df['name'] = df.name.astype(str).apply(lambda x: x.lower())
df['genres'] = df.genres.astype(str).apply(lambda x: x.lower())
df.sample()

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,speechiness,tempo,valence,year,genres
151402,0.239,michael bolton,0.611,280707,0.371,0,34IrFGwBby3nYswQOb7FEz,0.0,0,0.0762,-11.699,1,we're not makin' love anymore (with patti labelle),37,0.0323,121.992,0.156,1991,"['adult standards', 'soft rock']"


In [ ]:
df.sample()

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,speechiness,tempo,valence,year,genres
30586,0.0107,rod stewart,0.716,325813,0.465,0,3wsPg2KrRYZFi0inIFa41x,0.000667,2,0.099,-15.205,0,do ya think i'm sexy,51,0.0307,112.087,0.877,1978,"['mellow gold', 'soft rock']"


## Baseline Model (Benchmark)

####Model A
**Results**- Year seems to be the dominent feature, as all outcomes are tightly clustered around the sample year. When multiple version of a song appear the first is used as the sample
- OrdinalEncoder
- No scaling
- NearestNeighbors
  > Indented block(algorithm='auto', leaf_size=30, metric='minkowski', metric_params=None, n_jobs=None, n_neighbors=6, p=2,
radius=1.0)


####Model B
**Results**- The songs sound similar. When multiple version of a song appear the first is used as the sample
- OrdinalEncoder
- StandardScaler
- NearestNeighbors
  > Indented block(algorithm='auto', leaf_size=30, metric='minkowski', metric_params=None, n_jobs=None, n_neighbors=6, p=2,
radius=1.0)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',  
                 metric_params=None, n_jobs=None, n_neighbors=6, p=2,
                 radius=1.0)

#####Transformers

In [ ]:
# Encode catagorical features
encoder = ce.OrdinalEncoder(cols=['artists', 'name', 'genres', 'id'])
encoder.fit(df)
print('')

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [ ]:
df_e = encoder.transform(df)
df_e.dtypes

acousticness        float64
artists               int64
danceability        float64
duration_ms           int64
energy              float64
explicit              int64
id                    int64
instrumentalness    float64
key                   int64
liveness            float64
loudness            float64
mode                  int64
name                  int64
popularity            int64
speechiness         float64
tempo               float64
valence             float64
year                  int64
genres                int64
dtype: object

In [ ]:
# Scale features
scaler = preprocessing.StandardScaler()
df_s = scaler.fit_transform(df_e)

#####Initial Song

In [ ]:
# test input data
test_data = {
  "artists": 'Johnny Cash',
  "name": 'Ring of Fire'
}
test_data['artists'] = test_data['artists'].lower()
test_data['name'] = test_data['name'].lower()

# # test input data
# test_data = {
#   "id": '45XnLMuqf3vRfskEAMUeCH'}

In [ ]:
test_row=df[df['artists'].str.contains(test_data['artists']) & df['name'].str.contains(test_data['name'])]
test_row
# test_row=df[df['id'].str.contains(test_data['id'])]
# test_row

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,speechiness,tempo,valence,year,genres
8468,0.6230,johnny cash,0.659,158427,0.585,0,6YffUZJ2R06kyxyK6onezL,0.000213,7,0.348,-8.189,1,ring of fire,70,0.0288,104.111,0.784,1963,"['arkansas country', 'outlaw country']"
119074,0.0539,johnny cash,0.727,165173,0.805,0,5hSsg5rvkMxMScPKMESEUe,0.294000,2,0.340,-8.194,1,ring of fire - 1988 version,33,0.0301,111.070,0.826,1988,"['arkansas country', 'outlaw country']"
146865,0.3760,johnny cash,0.599,126907,0.585,0,1Ymfd9uTm3WveSwIXOSqOe,0.000287,8,0.379,-10.694,1,"ring of fire (with the carter family) - live at san quentin state prison, san quentin, ca - february 1969",23,0.0325,122.896,0.835,1969,"['arkansas country', 'outlaw country']"
161901,0.3310,johnny cash,0.384,308467,0.730,0,54Ws56PDXVIQZhu3TLlf5p,0.001620,5,0.452,-8.879,1,"folsom prison blues/i walk the line/ring of fire/the rebel-johnny yuma (with the carter family, the statler brothers & carl perkins) - live at san quentin state prison, san quentin, ca - february ...",23,0.1010,131.257,0.700,1969,"['arkansas country', 'outlaw country']"


In [ ]:
test_e = encoder.transform(test_row)
test_s = scaler.transform(test_e)
test_s

array([[ 0.31915629, -0.74613567,  0.69802956, -0.50446163,  0.38234864,
        -0.27001909, -1.56172192, -0.58489632,  0.51042804,  0.75613714,
         0.6307072 ,  0.64975212, -1.28281798,  2.01558189, -0.4211696 ,
        -0.42345476,  0.97866794, -0.50852546, -0.67284791],
       [-1.18087192, -0.74613567,  1.0847052 , -0.45862547,  1.19107844,
        -0.27001909,  0.66292201,  0.29741888, -0.91089826,  0.71185527,
         0.62982847,  0.64975212,  0.9224119 ,  0.32350986, -0.41406782,
        -0.19371464,  1.13752368,  0.42750279, -0.67284791],
       [-0.33188389, -0.74613567,  0.35684517, -0.71862646,  0.38234864,
        -0.27001909,  1.22188878, -0.58467408,  0.7946933 ,  0.9277294 ,
         0.19046798,  0.64975212,  1.43605394, -0.13380691, -0.40095686,
         0.19670159,  1.1715642 , -0.28387868, -0.67284791],
       [-0.45049445, -0.74613567, -0.86573223,  0.51499554,  0.9153751 ,
        -0.27001909,  1.52431131, -0.58067075, -0.05810248,  1.33180149,
         0.509

#####Wrangle

In [ ]:
def wrangle(X, encoder, scaler, spotify_df):
  X['artists'] = X['artists'].lower()
  X['name'] = X['name'].lower()
  result = df[spotify_df['artists'].str.contains(X['artists']) & spotify_df['name'].str.contains(X['name'])]
  X = encoder.transform(result)
  X = scaler.transform(X)
  return(X)

In [ ]:
my_song = wrangle(test_data, encoder, scaler, df)

#####Base Model

In [ ]:
# Simple baseline model
knn = NearestNeighbors(n_neighbors=6).fit(df_s)

# # Simple baseline model
# knn = NearestNeighbors(n_neighbors=6)
# knn.fit(df_s)

In [ ]:
def get_pred(my_song, spotify_df):
  results = knn.kneighbors(my_song)
  results = [results[1][0]]
  results = spotify_df.iloc[results[0]]
  return(results)

In [ ]:
prediction = get_pred(my_song, df)
prediction

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,speechiness,tempo,valence,year,genres
8468,0.623,johnny cash,0.659,158427,0.585,0,6YffUZJ2R06kyxyK6onezL,0.000213,7,0.348,-8.189,1,ring of fire,70,0.0288,104.111,0.784,1963,"['arkansas country', 'outlaw country']"
12073,0.620,billy squier,0.670,217613,0.591,0,52KvuGmgcgRdrLMXOtda0E,0.000000,7,0.373,-6.012,1,the stroke - remastered,66,0.0646,91.439,0.800,1981,"['album rock', 'classic rock', 'country rock', 'glam metal', 'hard rock', 'heartland rock', 'mellow gold', 'rock', 'soft rock']"
7863,0.733,ella fitzgerald,0.579,131733,0.502,0,65irrLqfCMRiO3p87P4C0D,0.000000,8,0.281,-7.570,1,frosty the snowman,74,0.0513,76.816,0.836,1960,"['adult standards', 'jazz blues', 'swing', 'vocal jazz']"
9285,0.365,the beatles,0.705,164107,0.515,0,2RnPATK99oGOZygnD2GTO6,0.000000,4,0.389,-9.343,1,with a little help from my friends - remastered 2009,65,0.0337,112.035,0.803,1967,"['beatlesque', 'british invasion', 'classic rock', 'merseybeat', 'psychedelic rock', 'rock']"
8712,0.623,solomon burke,0.733,154907,0.624,0,0sDeU2murnLh4yVHQ5IV70,0.000000,4,0.407,-9.849,1,cry to me,61,0.0374,114.586,0.912,1964,"['classic soul', 'funk', 'motown', 'soul', 'soul blues', 'southern soul']"
8286,0.623,solomon burke,0.733,154907,0.624,0,1tYk5zABu6h8CMP1cSea56,0.000000,4,0.407,-9.849,1,cry to me,59,0.0374,114.586,0.912,1962,"['classic soul', 'funk', 'motown', 'soul', 'soul blues', 'southern soul']"


Create joblibs for api to use for practice.


In [ ]:
import joblib
from joblib import dump, load

dump(encoder, 'base_encoder.joblib')
base_encoder = load('base_encoder.joblib')

dump(scaler, 'base_scaler.joblib')
base_scaler = load('base_scaler.joblib')

dump(get_pred, 'base_pred.joblib')
base_pred = load('base_pred.joblib')

In [ ]:
# dummy data
test_1 = {
    'artists':'peacock',
    'name':'pony'
}

In [ ]:
# test
tidy=wrangle(test_1, encoder, scaler, df)
answer=base_pred(tidy, df)
answer

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,speechiness,tempo,valence,year,genres
99902,0.2130,annette peacock,0.682,377627,0.430,0,3Cioq9gEBcdBZi6SAIfR8l,0.182000,0,0.0901,-10.946,1,pony,35,0.0572,134.496,0.628,1972,['not specified']
118318,0.0107,waterfront home,0.766,338800,0.446,0,0KoyYHBXh8cFC2xHjxz57f,0.000169,2,0.0620,-12.813,1,take a chance,38,0.0659,129.139,0.694,1984,['not specified']
50262,0.1720,the cast of grease 2,0.768,311627,0.602,0,1nzfWI8dJn1oExRv9thVw6,0.002790,0,0.0767,-8.740,1,who's that guy?,39,0.0313,128.439,0.671,1982,['not specified']
100697,0.0276,cloud one,0.553,271573,0.523,0,4qyrNAJUxJKRpYlTIZRL5O,0.107000,0,0.0982,-14.813,1,dust to dust,30,0.0345,150.827,0.698,1976,['not specified']
66710,0.0854,bill wyman,0.757,359893,0.554,0,22TBatKbsD4NNYqwAeQCH1,0.009420,2,0.0925,-11.227,1,"(si, si) je suis un rock star",40,0.0675,116.958,0.881,1974,['not specified']
98309,0.4810,augustin lehfuss,0.635,172000,0.527,0,2C6klKDE66IJsIpNQpDqaM,0.000000,0,0.1980,-7.824,1,love,21,0.0559,108.741,0.650,1964,['not specified']


## Modeling Approach #1

## Modeling Approach #2

## Hyper-parameter Tuning for Best Model

## Save Final Model

In [ ]:
import joblib

joblib.dump(model, '../models/model_name.joblib')